In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('../utils')
import database as db

connect to CVEFIXes


### safe/unsafe in commit locality

In [2]:
df = pd.read_sql("SELECT cve_id, hash, num_lines_added, num_lines_deleted,num_files FROM commits", con=db.conn)
df.drop_duplicates(subset=['cve_id', 'hash'], keep='first', inplace=True)
df['num_lines_added'] = df['num_lines_added'].apply(lambda x: int(x))
df['num_lines_deleted'] = df['num_lines_deleted'].apply(lambda x: int(x))
df['num_files'] = df['num_files'].apply(lambda x: int(x))
def print_statistics(col):
    print(f"# average {col}: " + str(df[col].sum()/len(df)))
    print(f"# median {col}: " + str(df[col].median()))
    print(f"# minimum {col}: " + str(df[col].min()))
    print(f"# maximum {col}: " + str(df[col].max()))
    print(f"# toatl {col}: " + str(df[col].sum()))
    print(f"std of {col}: " + str(df[col].std()))
    print("==================================")
print_statistics('num_lines_added')
print_statistics('num_lines_deleted')
print_statistics('num_files')

df = pd.read_sql("SELECT * FROM vul_safe_unsafe", con=db.conn)
df["unsafe_trait"] = float('nan')
df_manual = pd.read_sql("SELECT * FROM vul_safe_unsafe_manual2", con=db.conn)
df = df.append(df_manual, ignore_index=True)

df['safe_func_fix'] = df.apply(lambda x: max(float(x.safe_func_fix), float(x.safe_func)), axis=1)
df['unsafe_func_fix'] = df.apply(lambda x: max(float(x.unsafe_func_fix), float(x.unsafe_func)), axis=1)
df['unsafe_block_fix'] = df.apply(lambda x: max(float(x.unsafe_block_fix), float(x.unsafe_block)), axis=1)
print_statistics('safe_func_fix')
print_statistics('unsafe_func_fix')
print_statistics('unsafe_block_fix')

# average num_lines_added: 41.13084112149533
# median num_lines_added: 14.0
# minimum num_lines_added: 1
# maximum num_lines_added: 665
# toatl num_lines_added: 8802
std of num_lines_added: 81.96851383502117
# average num_lines_deleted: 18.16822429906542
# median num_lines_deleted: 4.0
# minimum num_lines_deleted: 0
# maximum num_lines_deleted: 330
# toatl num_lines_deleted: 3888
std of num_lines_deleted: 39.74637574201591
# average num_files: 1.8457943925233644
# median num_files: 1.0
# minimum num_files: 1
# maximum num_files: 14
# toatl num_files: 395
std of num_files: 1.9499743768068212
# average safe_func_fix: 3.8529411764705883
# median safe_func_fix: 1.0
# minimum safe_func_fix: 0.0
# maximum safe_func_fix: 83.0
# toatl safe_func_fix: 786.0
std of safe_func_fix: 9.122480281112225
# average unsafe_func_fix: 0.1568627450980392
# median unsafe_func_fix: 0.0
# minimum unsafe_func_fix: 0.0
# maximum unsafe_func_fix: 4.0
# toatl unsafe_func_fix: 32.0
std of unsafe_func_fix: 0.52980334

### safe/unsafe in commit locality across type

In [3]:
df_vul = pd.read_sql("SELECT id, package, sfp_id, published, severity FROM cve", con=db.conn)
df = pd.read_sql("SELECT * FROM vul_safe_unsafe", con=db.conn)
df["unsafe_trait"] = 0
df_manual = pd.read_sql("SELECT * FROM vul_safe_unsafe_manual2", con=db.conn)
df_data = df.append(df_manual, ignore_index=True)

df_files = pd.read_sql("SELECT cve_id, hash, num_files FROM commits", con=db.conn)
df_data['safe_function'] = df_data.apply(lambda x: max(float(x.safe_func_fix), float(x.safe_func)), axis=1)
df_data['unsafe_function'] = df_data.apply(lambda x: max(float(x.unsafe_func_fix), float(x.unsafe_func)), axis=1)
df_data['unsafe_block'] = df_data.apply(lambda x: max(float(x.unsafe_block_fix), float(x.unsafe_block)), axis=1)
df_data['unsafe_trait'] = df_data['unsafe_trait'].apply(lambda x: float(x))
def get_cat(x, cat):
    temp = eval(df_vul[df_vul['id']==x]['sfp_id'].values[0]) if len(df_vul[df_vul['id']==x]['sfp_id'].values)>0 else []
    if cat in temp:
        return True
    return False
cats = ['Memory Access', 'Memory Management', 'Synchronization', 'Tainted Input', 'Resource Management', 'Exception Management', 'Path Resolution']
for cat in cats:
    df = df_data[df_data["cve_id"].apply(lambda x: get_cat(x, cat))]
    
    def print_statistics(col):
        print(f"# average {col}: " + str(df[col].sum()/len(df)))
        print(f"# median {col}: " + str(df[col].median()))
        print(f"# minimum {col}: " + str(df[col].min()))
        print(f"# maximum {col}: " + str(df[col].max()))
        print(f"# toatl {col}: " + str(df[col].sum()))
        print(f"std of {col}: " + str(df[col].std()))
        print("==================================")
    print("Commit Locality Across "+cat)
    print_statistics('safe_function')
    print_statistics('unsafe_function')
    print_statistics('unsafe_block')
    print_statistics('unsafe_trait')
    print(len(df))
    print("\n")
    

Commit Locality Across Memory Access
# average safe_function: 3.1666666666666665
# median safe_function: 1.0
# minimum safe_function: 0.0
# maximum safe_function: 59.0
# toatl safe_function: 190.0
std of safe_function: 9.20237011965988
# average unsafe_function: 0.23333333333333334
# median unsafe_function: 0.0
# minimum unsafe_function: 0.0
# maximum unsafe_function: 3.0
# toatl unsafe_function: 14.0
std of unsafe_function: 0.5928004971139372
# average unsafe_block: 1.7333333333333334
# median unsafe_block: 0.0
# minimum unsafe_block: 0.0
# maximum unsafe_block: 50.0
# toatl unsafe_block: 104.0
std of unsafe_block: 6.614997512501079
# average unsafe_trait: 0.4
# median unsafe_trait: 0.0
# minimum unsafe_trait: 0.0
# maximum unsafe_trait: 4.0
# toatl unsafe_trait: 24.0
std of unsafe_trait: 0.9126062905469662
60


Commit Locality Across Memory Management
# average safe_function: 5.235294117647059
# median safe_function: 1.0
# minimum safe_function: 0.0
# maximum safe_function: 59.0
# to

### Fix patterns

In [4]:
df_auto = pd.read_sql("SELECT * FROM vul_safe_unsafe", con=db.conn)
df_manual = pd.read_sql("SELECT * FROM vul_safe_unsafe_manual2", con=db.conn)
df = df_auto.append(df_manual).reset_index()

In [5]:
# vulnerability locality: safe function
df_safe = df
df_safe = df_safe[df_safe["safe_func"]!=0]
print(len(df_safe))
add = len(df_safe[df_safe.apply(lambda x: True if x.safe_func < x.safe_func_fix else False, axis=1)])
remove = len(df_safe[df_safe.apply(lambda x: True if x.safe_func > x.safe_func_fix else False, axis=1)])
modified = len(df_safe[df_safe.apply(lambda x: True if x.safe_func == x.safe_func_fix else False, axis=1)])
print(add)
print(remove)
print(modified)

157
28
12
117


In [6]:
df_unsafe = df
df_unsafe = df_unsafe[df_unsafe["unsafe_func"]!=0]
# df_unsafe = df_unsafe[df_unsafe["unsafe_block"]==0]
print(len(df_unsafe))
add = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_func < x.unsafe_func_fix else False, axis=1)])
modified = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_func ==x.unsafe_func_fix else False, axis=1)])
remove = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_func > x.unsafe_func else False, axis=1)])
print(add)
print(remove)
print(modified)

20
0
0
16


In [7]:
df_unsafe = df
df_unsafe = df_unsafe[df_unsafe["unsafe_block"]!=0]
# df_unsafe = df_unsafe[df_unsafe["unsafe_block"]==0]
print(len(df_unsafe))
add = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_block < x.unsafe_block_fix else False, axis=1)])
modified = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_block == x.unsafe_block_fix else False, axis=1)])
remove = len(df_unsafe[df_unsafe.apply(lambda x: True if x.unsafe_block > x.unsafe_block_fix else False, axis=1)])
print(add)
print(remove)
print(modified)

68
5
27
36


In [8]:
from statsmodels.stats.proportion import proportion_confint
print(proportion_confint(32, 40, method='agresti_coull', alpha=0.05))

(0.6498552554829977, 0.8975717837986303)
